In [1]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 10.7 MB/s 
     |████████████████████████████████| 3.3 MB 29.3 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 895 kB 37.7 MB/s 
     |████████████████████████████████| 56 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [34]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np

MODEL = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

text = ["Good night honey 😊", "lmao xd 😊"]
#text = preprocess(text)

# Pytorch
roberta = AutoModel.from_pretrained(MODEL).to(device)
modules = [roberta.embeddings, *roberta.encoder.layer[:10]] #do przetestowania różne parametry
for module in modules:
    for param in module.parameters():
        param.requires_grad = False
#encoded_input = tokenizer(text, return_tensors='pt', padding=True)
#features = model(**encoded_input)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


df = pd.read_csv("train_dataset.csv")
X = df["tweet_content"].tolist()
X = [preprocess(x) for x in X]
y = (df["sarcasm_label"] == 'sarcastic').tolist()


In [36]:
from torch.nn.functional import pad
X_encoded  = tokenizer(X, return_tensors='pt', padding=True)['input_ids']

In [37]:
X_train, X_cross, y_train, y_cross = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
y_train = torch.Tensor(y_train).type(torch.long)
y_cross = torch.Tensor(y_cross).type(torch.long)


In [38]:
#dobija do f_score = 0.4 gdy trenujemy roberte
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self ):
        super().__init__()
        self.roberta = roberta
        self.lstm = nn.LSTM(768, 64, 2, bidirectional = True, dropout = 0.2)
        self.linear1 = nn.Linear(716, 100)
        self.linear2 = nn.Linear(100, 20)
        self.linear3 = nn.Linear(20, 2)
        self.pooling = nn.MaxPool2d((30, 5))
        self.softmax = nn.Softmax(dim = 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, x):
        x1 = self.roberta(x).last_hidden_state
        x, _ =  self.lstm(x1)
        x = torch.cat((x1, x), dim = 2)
        x = self.pooling(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [39]:
from torch.utils.data import TensorDataset, DataLoader

dataset_train = TensorDataset(X_train, y_train) # create your datset
dataloader_train = DataLoader(dataset_train,batch_size=40) # create your dataloader
dataset_cross = TensorDataset(X_cross, y_cross) # create your datset
dataloader_cross = DataLoader(dataset_cross,batch_size=40) # create your dataloader

In [40]:
def train(dataloader, model, loss_fn, optimizer): #fragment z dokumentacji pytorch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss :>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    print(f"Avg train loss: {train_loss:>8f} \n")


In [41]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    true_positive, true_negative, false_negative, false_positive = 0.0001, 0.0001, 0.0001, 0.0001 #zeby dzielenia przez zero nie bylo
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            y_pred = pred.argmax(1)
            correct += (y_pred == y).type(torch.long).sum().item()

            true_positive += torch.logical_and(y_pred == y, y == 1).sum().item()
            true_negative += torch.logical_and(y_pred == y, y == 0).sum().item()
            false_negative += torch.logical_and(y_pred != y, y == 1).sum().item() 
            false_positive += torch.logical_and(y_pred != y, y == 0).sum().item()
        

    recall = true_positive/(true_positive + false_negative)
    precision = true_positive/(true_positive + false_positive)
    specificity = true_negative/(false_positive + true_negative)
    f_score = (2*precision*recall)/(precision + recall)
    g_mean = (recall*specificity)**(0.5)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"\n recall: {recall} \n precision: {precision} \n specificity: {specificity} \n f_score: {f_score} \n g_mean: {g_mean}\n")

In [42]:
model = Model().to(device)

In [43]:
loss_fn = nn.CrossEntropyLoss(weight = torch.Tensor([1.0, 1.0 ])).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [45]:
epochs = 20
for t in range(epochs):
    train(dataloader_train, model, loss_fn, optimizer)
    test(dataloader_cross, model, loss_fn)
print("Done!")

loss: 0.305836  [    0/ 2260]
loss: 0.369402  [ 2000/ 2260]
Avg train loss: 0.319527 

Test Error: 
 Accuracy: 40.7%, Avg loss: 0.723447 


 recall: 0.7380946712031637 
 precision: 0.16533351182212705 
 specificity: 0.34927241194494313 
 f_score: 0.270152705749276 
 g_mean: 0.5077362563918779

loss: 0.331267  [    0/ 2260]
loss: 0.384424  [ 2000/ 2260]
Avg train loss: 0.304318 

Test Error: 
 Accuracy: 38.2%, Avg loss: 0.766578 


 recall: 0.7738088718836383 
 precision: 0.16455713186980667 
 specificity: 0.31392939129755043 
 f_score: 0.27139893828898687 
 g_mean: 0.4928705186264185

loss: 0.226590  [    0/ 2260]
loss: 0.472294  [ 2000/ 2260]
Avg train loss: 0.285544 

Test Error: 
 Accuracy: 37.5%, Avg loss: 0.815355 


 recall: 0.7857136054437965 
 precision: 0.1645886959657377 
 specificity: 0.3035343852247879 
 f_score: 0.2721651363586505 
 g_mean: 0.4883555018540636

loss: 0.278900  [    0/ 2260]
loss: 0.360671  [ 2000/ 2260]
Avg train loss: 0.259243 

Test Error: 
 Accuracy: 35.